In [ ]:
%pip install "ray[serve]"

In [ ]:
from focoos import Focoos, FocoosEnvHostUrl
from pprint import pprint
import os

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

pprint(focoos.list_models())

In [ ]:
import requests
import time


def ray_infer(model_id: str):
    image_path = "./assets/ade_val_034.jpg"
    im_bytes = open(image_path, "rb").read()
    response = requests.post(
        "http://localhost:8005",
        data=im_bytes,
        headers={"serve_multiplexed_model_id": str(model_id)},
    )
    return response.json()


t0 = time.time()
res = ray_infer("focoos_object365")
t1 = time.time()
print(f"t: {t1-t0:.3f} seconds")
print(res)

In [ ]:
iters = 10

print("RAY")
for model_id in [
    "645b15a6b2c044c2",
    "focoos_rtdetr",
    "focoos_ade_medium",
    "focoos_object365",
    "focoos_aeroscapes",
    "focoos_ade_small",
    "focoos_isaid_nano",
    "focoos_coco_tiny",
]:
    t0 = time.time()
    for i in range(iters):
        ray_infer(model_id)
    t1 = time.time()
    print(
        f"{model_id} fps: {iters/(t1-t0):.3f}  ({(t1-t0)*1000/iters:.3f}ms per image)"
    )

In [ ]:
import logging
from focoos import Focoos, FocoosEnvHostUrl
from pprint import pprint
import os

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
iters = 10
focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)
image_path = "./assets/ade_val_034.jpg"

print("API+TRITON+WAF")
prints = []
for model_id in [
    "focoos_rtdetr",
    "focoos_ade_medium",
    "focoos_object365",
    "focoos_aeroscapes",
    "focoos_ade_small",
    "focoos_isaid_nano",
]:
    model = focoos.get_model(model_id)

    t0 = time.time()
    for i in range(iters):
        model.remote_infer(image_path)
    t1 = time.time()
    _str = (
        f"{model_id} fps: {iters/(t1-t0):.3f}  ({(t1-t0)*1000/iters:.3f}ms per image)"
    )
    prints.append(_str)

print("\n".join(prints))

In [ ]:
import ray

ray.init(address="localhost:6379")

## RAY SERVE DEPLOY 

In [ ]:
!serve deploy ../ray-config.yaml -a http://127.0.0.1:8265

In [ ]:
!serve status

In [ ]:
!serve config


In [ ]:
!serve shutdown -y

## RAY JOBS

In [ ]:
from ray.job_submission import JobSubmissionClient
from ray.runtime_env import RuntimeEnv
import os


client = JobSubmissionClient("http://localhost:8265")
runtime_env = RuntimeEnv(
    # container={"image": "ghcr.io/focoosai/deeplearning:base-cu12-pt22"},
    pip=[
        "focoos",
        "requests",
        "Pillow~=10.4.0",
        "supervision~=0.24.0",
        "pydantic~=2.8.2",
        "pydantic-settings~=2.5.0",
        "tqdm~=4.66.6",
        "numpy~=1.26.4",
        "scipy~=1.14.1",
        "gradio",
    ],
    env_vars={"FOCOOS_API_KEY": os.environ["FOCOOS_API_KEY"]},
    # working_dir="./assets/"
)
for model_id in [
    "focoos_object365",
    "focoos_rtdetr",
    "focoos_ade_medium",
    "focoos_ade_small",
    "focoos_isaid_nano",
]:
    job_id = client.submit_job(
        entrypoint=f"python /app/apps/benchmark.py --model {model_id}",
        runtime_env=runtime_env,
        entrypoint_num_gpus=0.3,
        submission_id=f"{model_id}_benchmark_{int(time.time())}",
    )
    print(client.get_job_info(job_id))
    print(client.get_job_logs(job_id))
    print(client.get_job_status(job_id))

In [ ]:
print(client.get_job_status(job_id))
print(client.get_job_logs(job_id))

In [ ]:
client.get_job_info(job_id)

In [ ]:
client.delete_job("03000000")